In [6]:
import pandas as pd
import numpy as np

data = pd.read_csv('winequality-red.csv')

data['quality'] = 1*data['quality'].apply(lambda x: True if(x>6) else False,)

#min max scaling
#X = X-Xmin/Xmax-Xmin
l = (data.drop('quality',axis=1)).columns
for i in l:
    data[i] = data[i].apply(lambda x: (x-min(data[i]))/(max(data[i])-min(data[i])))
data.to_csv('Logistic_regression.csv')
# MAKING THE DATA SET FOR ID3 ALGORITHM
data_ = pd.read_csv('winequality-red.csv')
for i in range(len(data_['quality'])):
    if data_['quality'].iloc[i] < 5:
        data_['quality'].iloc[i]=0
    elif ((data_['quality'].iloc[i] == 5) or (data_['quality'].iloc[i] == 6)):
        data_['quality'].iloc[i] = 1
    else:
        data_['quality'].iloc[i]=2
#NORMALIZING THE DATA 
for i in l:
    data_[i]=data_[i].apply(lambda x: (x-data_[i].mean())/np.std(data_[i]))
#DIVIDING DATA INTO BINS 0,1,2,3
for i in l:
    bin_size = (max(data_[i]) - min(data_[i]))/4
    start = min(data_[i])
    b_1 = start + bin_size
    b_2 = b_1+bin_size
    b_3 = b_2+bin_size
    end = max(data_[i])
    for j in range(len(data_[i])):
        if data_[i].iloc[j]>=start and data_[i].iloc[j]<b_1:
            data_[i].iloc[j] = 0
        elif data_[i].iloc[j]>=b_1 and data_[i].iloc[j]<b_2:
            data_[i].iloc[j] = 1
        elif data_[i].iloc[j]>=b_2 and data_[i].iloc[j]<b_3:
            data_[i].iloc[j] = 2
        else:
            data_[i].iloc[j] = 3
data_.to_csv('ID3_dataset.csv')


C:\Users\udit1\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [3]:
import pandas as pd
import numpy as np
#implementing logistic regression

thresh_hold = 0.0000001#Adjust this for faster processing !
def Gradient_Descent(X_train,y,theta):
	lin_est = np.dot(np.transpose(theta),X_train) ## computing here the h(x)
	t_1 = np.dot(np.transpose(y),np.log(sigmoid(lin_est))) ##computing y*log(h(x))
	t_2 = np.dot(np.transpose(1-y),np.log(1-sigmoid(lin_est)))##computing (1-y)*h(x)
	err = -(t_1+t_2) 
	cost= err/np.shape(X_train)[1] ##the error term
	print('Starting Cost theta initiated as one:',cost)
	old = cost
	diff = cost
	while(diff >= thresh_hold):
		grad = np.dot(X_train,(sigmoid(lin_est)-y))
		grad = grad/np.shape(X_train)[1]
		alpha = 0.09
		theta = theta - alpha*grad
		lin_est = np.dot(np.transpose(theta),X_train)
		t_1 = np.dot(np.transpose(y),np.log(sigmoid(lin_est)))
		t_2 = np.dot(np.transpose(1-y),np.log(1-sigmoid(lin_est)))
		err = -(t_1+t_2)
		cost= err/np.shape(X_train)[1]
#		print('Iter_Cost: ',cost)
		diff = abs(cost-old)
		old = cost
	return(theta,cost,lin_est)

def sigmoid(x):
    sigmoid = 1/(1+np.exp(-x))
    return(sigmoid)

def binary_func(pred_og):
    for i in range(len(pred_og)):
        if pred_og[i]>0.5:
            pred_og[i]=1
        else:
            pred_og[i]=0
    return(pred_og)
data = pd.read_csv('Logistic_regression.csv')
data.drop('Unnamed: 0',axis=1,inplace=True)
print('Implementing Logistic Regression')
l = (data.drop('quality',axis=1)).columns
feature_list = l 
feature = (data[feature_list].values) 
theta = np.ones(len(feature_list)+1) #INITIATING THETA
a=np.ones(len(feature)) #ADDING X(0)
X_train = np.insert(feature,0,a,axis=1) #ADDING X(0)
X_train =np.transpose(X_train)
y = np.array(data['quality'])
theta,cost,lin_est = Gradient_Descent(X_train,y,theta)
pred_og = sigmoid(lin_est)
pred_og = binary_func(pred_og)
print('The metrics for logistic regression using my implementation of logistic regression::')
from sklearn.metrics import classification_report
print(classification_report(y,pred_og))
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='saga',penalty='none')
lr.fit(data.drop('quality',axis=1),data['quality'])
from sklearn.metrics import classification_report

pred = lr.predict(data.drop('quality',axis=1))
print('The metrics for logistic regression using SAGA solver::')
print(classification_report(y,pred))


preds = pd.DataFrame(pred_og,columns=['predicted_by_me'])
preds['predicted_by_not_me'] = pred
error_rate = (preds['predicted_by_me'] != preds['predicted_by_not_me']).mean()

print('The prediction mismatch between my model and the SAGA solver model is ::',error_rate)
print('\n')
print('Implementing Cross-Validation')
print('\n')

##3 fold cross validation for my model begins::

slot_1 = data.iloc[0:533]
slot_2 = data.iloc[533:1066]
slot_3 = data.iloc[1066:1599]

precision=[]
recall=[]
accuracy=[]
for i in range(1,4):
    if i == 1:
        test_set = slot_1
        train_set = slot_2.append(slot_3)
        feature_list = l #USED LIST OF FEATURES GETS INCREASED EVERY ITERATION
        feature = (train_set[feature_list].values) #GETTING VALUES FROM DATA FRAME AS AN ARRAY
        theta = np.ones(len(feature_list)+1) #INITIATING THETA
        a=np.ones(len(feature)) #ADDING X(0)
        X_train = np.insert(feature,0,a,axis=1) #ADDING X(0)
        X_train =np.transpose(X_train)
        y = np.array(train_set['quality'])
        params,cost,lin_est = Gradient_Descent(X_train,y,theta)
        feature_test = (test_set[feature_list].values)
        theta = params #PARAMS ARE THE FINAL ITERATION PARAMETERS
        a=np.ones(len(feature_test)) #ADDING X(0)
        X_test = np.insert(feature_test,0,a,axis=1)
        X_test =np.transpose(X_test)
        y_test = np.array(test_set['quality'])
        test_err = np.dot(np.transpose(theta),X_test)
        temp = binary_func(sigmoid(test_err))
        from sklearn.metrics import classification_report 
        X =classification_report(y_test,temp,output_dict=True)
        precision.append(X['1']['precision'])
        recall.append(X['1']['recall'])
        accuracy.append(X['accuracy'])
    
        
            
    if i == 2:
        test_set = slot_2
        train_set = slot_1.append(slot_3)
        feature_list = l #USED LIST OF FEATURES GETS INCREASED EVERY ITERATION
        feature = (train_set[feature_list].values) #GETTING VALUES FROM DATA FRAME AS AN ARRAY
        theta = np.ones(len(feature_list)+1) #INITIATING THETA
        a=np.ones(len(feature)) #ADDING X(0)
        X_train = np.insert(feature,0,a,axis=1) #ADDING X(0)
        X_train =np.transpose(X_train)
        y = np.array(train_set['quality'])
        params,cost,lin_est = Gradient_Descent(X_train,y,theta)
        feature_test = (test_set[feature_list].values)
        theta = params #PARAMS ARE THE FINAL ITERATION PARAMETERS
        a=np.ones(len(feature_test)) #ADDING X(0)
        X_test = np.insert(feature_test,0,a,axis=1)
        X_test =np.transpose(X_test)
        y_test = np.array(test_set['quality'])
        test_err = np.dot(np.transpose(theta),X_test)
        temp = binary_func(sigmoid(test_err))
        from sklearn.metrics import classification_report 
        X =classification_report(y_test,temp,output_dict=True)
        precision.append(X['1']['precision'])
        recall.append(X['1']['recall'])
        accuracy.append(X['accuracy'])
    if i == 3:
        test_set = slot_3
        train_set = slot_1.append(slot_2)
        feature_list = l #USED LIST OF FEATURES GETS INCREASED EVERY ITERATION
        feature = (train_set[feature_list].values) #GETTING VALUES FROM DATA FRAME AS AN ARRAY
        theta = np.ones(len(feature_list)+1) #INITIATING THETA
        a=np.ones(len(feature)) #ADDING X(0)
        X_train = np.insert(feature,0,a,axis=1) #ADDING X(0)
        X_train =np.transpose(X_train)
        y = np.array(train_set['quality'])
        params,cost,lin_est = Gradient_Descent(X_train,y,theta)
        feature_test = (test_set[feature_list].values)
        theta = params #PARAMS ARE THE FINAL ITERATION PARAMETERS
        a=np.ones(len(feature_test)) #ADDING X(0)
        X_test = np.insert(feature_test,0,a,axis=1)
        X_test =np.transpose(X_test)
        y_test = np.array(test_set['quality'])
        test_err = np.dot(np.transpose(theta),X_test)
        temp = binary_func(sigmoid(test_err))
        from sklearn.metrics import classification_report 
        X =classification_report(y_test,temp,output_dict=True)
        precision.append(X['1']['precision'])
        recall.append(X['1']['recall'])
        accuracy.append(X['accuracy'])
print('\n')
print('The metrics for a 3 fold cross validation on my logistic regression model')
print('precision:',sum(precision)/3)
print('recall:',sum(recall)/3)
print('accuracy:',sum(accuracy)/3)

from sklearn.model_selection import cross_val_score
l = (data.drop('quality',axis=1)).columns
precision_saga=cross_val_score(lr,data[l],data['quality'],cv=3,scoring = 'precision')
recall_saga=cross_val_score(lr,data[l],data['quality'],cv=3,scoring = 'recall')
accuracy_saga=cross_val_score(lr,data[l],data['quality'],cv=3,scoring = 'accuracy')
print('The metrics for a 3 fold cross validation on SAGA solver model')
print('\n')
print('precision:',sum(precision_saga)/3)
print('recall:',sum(recall_saga)/3)
print('accuracy:',sum(accuracy_saga)/3)

Implementing Logistic Regression
Starting Cost theta initiated as one: 3.388591156716365
The metrics for logistic regression using my implementation of logistic regression::
              precision    recall  f1-score   support

           0       0.90      0.97      0.93      1382
           1       0.64      0.32      0.43       217

    accuracy                           0.88      1599
   macro avg       0.77      0.65      0.68      1599
weighted avg       0.87      0.88      0.87      1599

The metrics for logistic regression using SAGA solver::
              precision    recall  f1-score   support

           0       0.90      0.97      0.94      1382
           1       0.64      0.35      0.45       217

    accuracy                           0.88      1599
   macro avg       0.77      0.66      0.69      1599
weighted avg       0.87      0.88      0.87      1599

The prediction mismatch between my model and the SAGA solver model is :: 0.0150093808630394


Implementing Cross-Val

C:\Users\udit1\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\udit1\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\udit1\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [4]:
import pandas as pd
import numpy as np
from numpy import log2 as log
data = pd.read_csv('ID3_dataset.csv')
data.drop('Unnamed: 0',axis=1,inplace=True)
#calculating entropy
def entropy_node(data):
    determine = data.keys()[-1]
    l = data[determine].unique()
    a = data[determine].value_counts()
    entropy = 0
    for i in l:
        prob = a[i]/len(data[determine])
        entropy = entropy-prob*log(prob)
    return(entropy)
def attribute_entropy(data,attribute):
    determine = data.keys()[-1]
    l = data[attribute].unique()
    attribute_entropy = 0
    for i in l:
        entropy_atr_class = entropy_node(data[data[attribute]==i])
        attribute_entropy += entropy_atr_class*len(data[data[attribute]==i][determine])/len(data[determine])
    return(attribute_entropy)

## finding the maximum gain = entropy_node - attribute entropy
def max_gain_class(data):
    determine = data.keys()[-1]
    attributes = data.keys()[:-1]
    info_gain = {}
    for i in attributes:
        temp = entropy_node(data) - attribute_entropy(data,i)
        info_gain[i] = temp
    winner_class = max(info_gain,key=info_gain.get)
    return(winner_class)

def decision_tree(data,tree=None):
    determine = data.keys()[-1]

    node = max_gain_class(data)
    #the components of the max gain class
    l = data[node].unique()
    if tree is None:
        tree = {}
        tree[node] ={}
    for i in l:
        sub_data = data[data[node]==i].reset_index(drop=True)
        y = len(sub_data.columns)
        if len(sub_data[determine])<=10 or y==2:#here y==2 stops the tree when only one feature is remaining in the feature set, as there will be no features in future iterations
            tree[node][i] = sub_data[determine].mode()[0]# assigns the mode of the remaining data to the leaf
        else:
            tree[node][i]=decision_tree(sub_data.drop(node,axis=1))#here we drop the current node feature from the sub_data and pass it for further split
    return(tree)
def predict(data,tree):#this is a code to traverse the tree dictionary
    temp = tree[list(tree.keys())[0]][data[list(tree.keys())[0]]]
    if (temp ==1) or (temp ==0) or (temp ==2):
        return(temp)
    else:
        tree =tree[list(tree.keys())[0]][data[list(tree.keys())[0]]]
        temp=predict(data,tree)
        return(temp)
# getting the tree
tree =decision_tree(data)
#getting the predictions
pred=[]
for i in range(len(data)):
    pred.append(predict(data.iloc[i],tree))
#Checking the performance of ID3 algorithm
print('The metrics for ID3 algorithm are::')
predictions = pd.DataFrame(pred,columns=['predictions'])
predictions['true'] = data['quality']
from sklearn.metrics import classification_report
print(classification_report(predictions['true'],predictions['predictions']))
#Computing the Decision Tree Classifier Using Scikitlearn
print('The metrics for scikit learn implementation with a min_samples_split =10:: ')
from sklearn.tree import DecisionTreeClassifier
dr = DecisionTreeClassifier(min_samples_split=10)
dr.fit(data.drop('quality',axis=1),data['quality'])
pred_=dr.predict(data.drop('quality',axis=1))
print(classification_report(predictions['true'],pred_))
from sklearn.model_selection import cross_val_score
l = (data.drop('quality',axis=1)).columns
precision_=cross_val_score(dr,data[l],data['quality'],cv=3,scoring = 'precision_macro')
recall_=cross_val_score(dr,data[l],data['quality'],cv=3,scoring = 'recall_macro')
accuracy_=cross_val_score(dr,data[l],data['quality'],cv=3,scoring = 'accuracy')
print('The metrics for a 3 fold cross validation on Decision tree classifier model')
print('\n')
print('Macro precision:',sum(precision_)/3)
print('Macro recall:',sum(recall_)/3)
print('accuracy:',sum(accuracy_)/3)



The metrics for ID3 algorithm are::
              precision    recall  f1-score   support

           0       0.64      0.37      0.46        63
           1       0.89      0.97      0.93      1319
           2       0.78      0.42      0.55       217

    accuracy                           0.87      1599
   macro avg       0.77      0.59      0.65      1599
weighted avg       0.86      0.87      0.86      1599

The metrics for scikit learn implementation with a min_samples_split =10:: 
              precision    recall  f1-score   support

           0       0.73      0.30      0.43        63
           1       0.90      0.97      0.93      1319
           2       0.76      0.53      0.62       217

    accuracy                           0.88      1599
   macro avg       0.80      0.60      0.66      1599
weighted avg       0.87      0.88      0.87      1599

The metrics for a 3 fold cross validation on Decision tree classifier model


Macro precision: 0.4942172223686427
Macro recall

In [7]:
import pandas as pd
import numpy as np
#implementing logistic regression

thresh_hold = 0.0000001#Adjust this for faster processing !
def Gradient_Descent(X_train,y,theta):
	lin_est = np.dot(np.transpose(theta),X_train) ## computing here the h(x)
	t_1 = np.dot(np.transpose(y),np.log(sigmoid(lin_est))) ##computing y*log(h(x))
	t_2 = np.dot(np.transpose(1-y),np.log(1-sigmoid(lin_est)))##computing (1-y)*h(x)
	err = -(t_1+t_2) 
	cost= err/np.shape(X_train)[1] ##the error term
	print('Starting Cost theta initiated as one:',cost)
	old = cost
	diff = cost
	while(diff >= thresh_hold):
		grad = np.dot(X_train,(sigmoid(lin_est)-y))
		grad = grad/np.shape(X_train)[1]
		alpha = 0.09
		theta = theta - alpha*grad
		lin_est = np.dot(np.transpose(theta),X_train)
		t_1 = np.dot(np.transpose(y),np.log(sigmoid(lin_est)))
		t_2 = np.dot(np.transpose(1-y),np.log(1-sigmoid(lin_est)))
		err = -(t_1+t_2)
		cost= err/np.shape(X_train)[1]
#		print('Iter_Cost: ',cost)
		diff = abs(cost-old)
		old = cost
	return(theta,cost,lin_est)

def sigmoid(x):
    sigmoid = 1/(1+np.exp(-x))
    return(sigmoid)

def binary_func(pred_og):
    for i in range(len(pred_og)):
        if pred_og[i]>0.5:
            pred_og[i]=1
        else:
            pred_og[i]=0
    return(pred_og)
data = pd.read_csv('Logistic_regression.csv')
data.drop('Unnamed: 0',axis=1,inplace=True)
print('Implementing Logistic Regression')
l = (data.drop('quality',axis=1)).columns
feature_list = l 
feature = (data[feature_list].values) 
theta = np.ones(len(feature_list)+1) #INITIATING THETA
a=np.ones(len(feature)) #ADDING X(0)
X_train = np.insert(feature,0,a,axis=1) #ADDING X(0)
X_train =np.transpose(X_train)
y = np.array(data['quality'])
theta,cost,lin_est = Gradient_Descent(X_train,y,theta)
pred_og = sigmoid(lin_est)
pred_og = binary_func(pred_og)
print('The metrics for logistic regression using my implementation of logistic regression::')
from sklearn.metrics import classification_report
print(classification_report(y,pred_og))
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='saga',penalty='none')
lr.fit(data.drop('quality',axis=1),data['quality'])
from sklearn.metrics import classification_report

pred = lr.predict(data.drop('quality',axis=1))
print('The metrics for logistic regression using SAGA solver::')
print(classification_report(y,pred))


preds = pd.DataFrame(pred_og,columns=['predicted_by_me'])
preds['predicted_by_not_me'] = pred
error_rate = (preds['predicted_by_me'] != preds['predicted_by_not_me']).mean()

print('The prediction mismatch between my model and the SAGA solver model is ::',error_rate)
print('\n')
print('Implementing Cross-Validation')
print('\n')

##3 fold cross validation for my model begins::

slot_1 = data.iloc[0:533]
slot_2 = data.iloc[533:1066]
slot_3 = data.iloc[1066:1599]

precision=[]
recall=[]
accuracy=[]
for i in range(1,4):
    if i == 1:
        test_set = slot_1
        train_set = slot_2.append(slot_3)
        feature_list = l #USED LIST OF FEATURES GETS INCREASED EVERY ITERATION
        feature = (train_set[feature_list].values) #GETTING VALUES FROM DATA FRAME AS AN ARRAY
        theta = np.ones(len(feature_list)+1) #INITIATING THETA
        a=np.ones(len(feature)) #ADDING X(0)
        X_train = np.insert(feature,0,a,axis=1) #ADDING X(0)
        X_train =np.transpose(X_train)
        y = np.array(train_set['quality'])
        params,cost,lin_est = Gradient_Descent(X_train,y,theta)
        feature_test = (test_set[feature_list].values)
        theta = params #PARAMS ARE THE FINAL ITERATION PARAMETERS
        a=np.ones(len(feature_test)) #ADDING X(0)
        X_test = np.insert(feature_test,0,a,axis=1)
        X_test =np.transpose(X_test)
        y_test = np.array(test_set['quality'])
        test_err = np.dot(np.transpose(theta),X_test)
        temp = binary_func(sigmoid(test_err))
        from sklearn.metrics import classification_report 
        X =classification_report(y_test,temp,output_dict=True)
        precision.append(X['1']['precision'])
        recall.append(X['1']['recall'])
        accuracy.append(X['accuracy'])
    
        
            
    if i == 2:
        test_set = slot_2
        train_set = slot_1.append(slot_3)
        feature_list = l #USED LIST OF FEATURES GETS INCREASED EVERY ITERATION
        feature = (train_set[feature_list].values) #GETTING VALUES FROM DATA FRAME AS AN ARRAY
        theta = np.ones(len(feature_list)+1) #INITIATING THETA
        a=np.ones(len(feature)) #ADDING X(0)
        X_train = np.insert(feature,0,a,axis=1) #ADDING X(0)
        X_train =np.transpose(X_train)
        y = np.array(train_set['quality'])
        params,cost,lin_est = Gradient_Descent(X_train,y,theta)
        feature_test = (test_set[feature_list].values)
        theta = params #PARAMS ARE THE FINAL ITERATION PARAMETERS
        a=np.ones(len(feature_test)) #ADDING X(0)
        X_test = np.insert(feature_test,0,a,axis=1)
        X_test =np.transpose(X_test)
        y_test = np.array(test_set['quality'])
        test_err = np.dot(np.transpose(theta),X_test)
        temp = binary_func(sigmoid(test_err))
        from sklearn.metrics import classification_report 
        X =classification_report(y_test,temp,output_dict=True)
        precision.append(X['1']['precision'])
        recall.append(X['1']['recall'])
        accuracy.append(X['accuracy'])
    if i == 3:
        test_set = slot_3
        train_set = slot_1.append(slot_2)
        feature_list = l #USED LIST OF FEATURES GETS INCREASED EVERY ITERATION
        feature = (train_set[feature_list].values) #GETTING VALUES FROM DATA FRAME AS AN ARRAY
        theta = np.ones(len(feature_list)+1) #INITIATING THETA
        a=np.ones(len(feature)) #ADDING X(0)
        X_train = np.insert(feature,0,a,axis=1) #ADDING X(0)
        X_train =np.transpose(X_train)
        y = np.array(train_set['quality'])
        params,cost,lin_est = Gradient_Descent(X_train,y,theta)
        feature_test = (test_set[feature_list].values)
        theta = params #PARAMS ARE THE FINAL ITERATION PARAMETERS
        a=np.ones(len(feature_test)) #ADDING X(0)
        X_test = np.insert(feature_test,0,a,axis=1)
        X_test =np.transpose(X_test)
        y_test = np.array(test_set['quality'])
        test_err = np.dot(np.transpose(theta),X_test)
        temp = binary_func(sigmoid(test_err))
        from sklearn.metrics import classification_report 
        X =classification_report(y_test,temp,output_dict=True)
        precision.append(X['1']['precision'])
        recall.append(X['1']['recall'])
        accuracy.append(X['accuracy'])
print('\n')
print('The metrics for a 3 fold cross validation on my logistic regression model')
print('precision:',sum(precision)/3)
print('recall:',sum(recall)/3)
print('accuracy:',sum(accuracy)/3)
print('\n')
from sklearn.model_selection import cross_val_score
l = (data.drop('quality',axis=1)).columns
precision_saga=cross_val_score(lr,data[l],data['quality'],cv=3,scoring = 'precision')
recall_saga=cross_val_score(lr,data[l],data['quality'],cv=3,scoring = 'recall')
accuracy_saga=cross_val_score(lr,data[l],data['quality'],cv=3,scoring = 'accuracy')
print('The metrics for a 3 fold cross validation on SAGA solver model')
print('precision:',sum(precision_saga)/3)
print('recall:',sum(recall_saga)/3)
print('accuracy:',sum(accuracy_saga)/3)

Implementing Logistic Regression
Starting Cost theta initiated as one: 3.388591156716365
The metrics for logistic regression using my implementation of logistic regression::
              precision    recall  f1-score   support

           0       0.90      0.97      0.93      1382
           1       0.64      0.32      0.43       217

    accuracy                           0.88      1599
   macro avg       0.77      0.65      0.68      1599
weighted avg       0.87      0.88      0.87      1599

The metrics for logistic regression using SAGA solver::
              precision    recall  f1-score   support

           0       0.90      0.97      0.94      1382
           1       0.64      0.35      0.45       217

    accuracy                           0.88      1599
   macro avg       0.77      0.66      0.69      1599
weighted avg       0.87      0.88      0.87      1599

The prediction mismatch between my model and the SAGA solver model is :: 0.0150093808630394


Implementing Cross-Val

C:\Users\udit1\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\udit1\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\udit1\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


The metrics for a 3 fold cross validation on SAGA solver model
precision: 0.5512121212121213
recall: 0.3094241501775748
accuracy: 0.8736801229697626


In [8]:
import pandas as pd
import numpy as np
from numpy import log2 as log
data = pd.read_csv('ID3_dataset.csv')
data.drop('Unnamed: 0',axis=1,inplace=True)
#calculating entropy
def entropy_node(data):
    determine = data.keys()[-1]
    l = data[determine].unique()
    a = data[determine].value_counts()
    entropy = 0
    for i in l:
        prob = a[i]/len(data[determine])
        entropy = entropy-prob*log(prob)
    return(entropy)
def attribute_entropy(data,attribute):
    determine = data.keys()[-1]
    l = data[attribute].unique()
    attribute_entropy = 0
    for i in l:
        entropy_atr_class = entropy_node(data[data[attribute]==i])
        attribute_entropy += entropy_atr_class*len(data[data[attribute]==i][determine])/len(data[determine])
    return(attribute_entropy)

## finding the maximum gain = entropy_node - attribute entropy
def max_gain_class(data):
    determine = data.keys()[-1]
    attributes = data.keys()[:-1]
    info_gain = {}
    for i in attributes:
        temp = entropy_node(data) - attribute_entropy(data,i)
        info_gain[i] = temp
    winner_class = max(info_gain,key=info_gain.get)
    return(winner_class)

def decision_tree(data,tree=None):
    determine = data.keys()[-1]

    node = max_gain_class(data)
    #the components of the max gain class
    l = data[node].unique()
    if tree is None:
        tree = {}
        tree[node] ={}
    for i in l:
        sub_data = data[data[node]==i].reset_index(drop=True)
        y = len(sub_data.columns)
        if len(sub_data[determine])<=10 or y==2:#here y==2 stops the tree when only one feature is remaining in the feature set, as there will be no features in future iterations
            tree[node][i] = sub_data[determine].mode()[0]# assigns the mode of the remaining data to the leaf
        else:
            tree[node][i]=decision_tree(sub_data.drop(node,axis=1))#here we drop the current node feature from the sub_data and pass it for further split
    return(tree)
def predict(data,tree):#this is a code to traverse the tree dictionary
    temp = tree[list(tree.keys())[0]][data[list(tree.keys())[0]]]
    if (temp ==1) or (temp ==0) or (temp ==2):
        return(temp)
    else:
        tree =tree[list(tree.keys())[0]][data[list(tree.keys())[0]]]
        temp=predict(data,tree)
        return(temp)
# getting the tree
tree =decision_tree(data)
#getting the predictions
pred=[]
for i in range(len(data)):
    pred.append(predict(data.iloc[i],tree))
#Checking the performance of ID3 algorithm
print('The metrics for ID3 algorithm are::')
predictions = pd.DataFrame(pred,columns=['predictions'])
predictions['true'] = data['quality']
from sklearn.metrics import classification_report
print(classification_report(predictions['true'],predictions['predictions']))
#Computing the Decision Tree Classifier Using Scikitlearn
print('The metrics for scikit learn implementation with a min_samples_split =10:: ')
from sklearn.tree import DecisionTreeClassifier
dr = DecisionTreeClassifier(min_samples_split=10)
dr.fit(data.drop('quality',axis=1),data['quality'])
pred_=dr.predict(data.drop('quality',axis=1))
print(classification_report(predictions['true'],pred_))
from sklearn.model_selection import cross_val_score
l = (data.drop('quality',axis=1)).columns
precision_=cross_val_score(dr,data[l],data['quality'],cv=3,scoring = 'precision_macro')
recall_=cross_val_score(dr,data[l],data['quality'],cv=3,scoring = 'recall_macro')
accuracy_=cross_val_score(dr,data[l],data['quality'],cv=3,scoring = 'accuracy')
print('The metrics for a 3 fold cross validation on Decision tree classifier model')
print('\n')
print('Macro precision:',sum(precision_)/3)
print('Macro recall:',sum(recall_)/3)
print('accuracy:',sum(accuracy_)/3)


The metrics for ID3 algorithm are::
              precision    recall  f1-score   support

           0       0.64      0.37      0.46        63
           1       0.89      0.97      0.93      1319
           2       0.78      0.42      0.55       217

    accuracy                           0.87      1599
   macro avg       0.77      0.59      0.65      1599
weighted avg       0.86      0.87      0.86      1599

The metrics for scikit learn implementation with a min_samples_split =10:: 
              precision    recall  f1-score   support

           0       0.73      0.30      0.43        63
           1       0.90      0.97      0.93      1319
           2       0.77      0.51      0.61       217

    accuracy                           0.88      1599
   macro avg       0.80      0.59      0.66      1599
weighted avg       0.87      0.88      0.87      1599

The metrics for a 3 fold cross validation on Decision tree classifier model


Macro precision: 0.514211608285181
Macro recall: